In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import glob
import pandas as pd

In [2]:
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
batch_size = 512
epochs = 20
learning_rate = 1e-3

In [25]:
house_label = {
    "Gryffindor":0,
    "Hufflepuff":1,
    "Ravenclaw":2,
    "Slytherin":3     
}

In [29]:
full_dataset = pd.read_csv("pics_by_house64x.csv", index_col=0)

X = full_dataset.drop(columns=['house'])
X = torch.tensor(X.values).float()
y = full_dataset['house'].apply(lambda x: house_label[x])
y = torch.tensor(y.values)

train_size, test_size, validation_size = 0.8, 0.1, 0.1

In [33]:
from torch.utils.data import Dataset

class myDataset(Dataset):
    def __init__(self, samples,labels):
        self.samples = samples
        self.labels = labels

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

In [34]:
data = myDataset(X, y)

In [35]:
train_data, test_data, validation_data = torch.utils.data.random_split(
    data,
    [
        int(train_size*full_dataset.shape[0]),
        int(test_size*full_dataset.shape[0]),
        full_dataset.shape[0]-(int(test_size*full_dataset.shape[0])+int(train_size*full_dataset.shape[0]))
    ],
    generator=torch.Generator().manual_seed(42)
)

In [36]:
data

In [37]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size, shuffle=True
)


In [38]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=512
        )
        self.encoder_output_layer = nn.Linear(
            in_features=512, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=512
        )
        self.decoder_output_layer = nn.Linear(
            in_features=512, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.sigmoid(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.sigmoid(activation)
        return reconstructed

In [39]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=4096).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()

In [40]:
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 4096).to(device)
        print(batch_features)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()      
        # compute reconstructions
        outputs = model(batch_features)
        print(outputs)
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        print(train_loss)
        # compute accumulated gradients
        train_loss.backward()
        # perform parameter update based on current gradients
        optimizer.step()
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    # compute the epoch training loss
    loss = loss / len(train_loader)
    print(loss)
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))

tensor([[126., 131., 144.,  ..., 106., 106., 125.],
        [125., 138., 144.,  ...,  95.,  91., 116.],
        [111.,  91., 103.,  ...,  92.,  92.,  98.],
        ...,
        [128., 127., 128.,  ..., 139., 130., 125.],
        [131., 137., 134.,  ...,  81.,  64.,  92.],
        [127., 135., 144.,  ..., 136., 143., 135.]], device='cuda:0')
tensor([[0.4785, 0.5002, 0.5917,  ..., 0.4923, 0.5009, 0.5174],
        [0.4995, 0.5220, 0.5594,  ..., 0.5033, 0.4682, 0.5151],
        [0.4996, 0.5145, 0.5842,  ..., 0.4996, 0.4651, 0.5406],
        ...,
        [0.4973, 0.5070, 0.5753,  ..., 0.4736, 0.4875, 0.5268],
        [0.5035, 0.5124, 0.5689,  ..., 0.4953, 0.4593, 0.5165],
        [0.4772, 0.5071, 0.5934,  ..., 0.5002, 0.4795, 0.5263]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor(19545.5684, device='cuda:0', grad_fn=<MseLossBackward>)
19545.568359375
epoch : 1/20, recon loss = 19545.56835938
tensor([[115.,  94.,  98.,  ...,  87.,  82., 105.],
        [160., 197., 188.,  ..., 22

        [ 96.,  68.,  79.,  ..., 142., 143., 136.]], device='cuda:0')
tensor([[0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477],
        [0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477],
        [0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477],
        ...,
        [0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477],
        [0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477],
        [0.8680, 0.9516, 0.9608,  ..., 0.9345, 0.9345, 0.9477]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor(19438.2363, device='cuda:0', grad_fn=<MseLossBackward>)
19438.236328125
epoch : 11/20, recon loss = 19438.23632812
tensor([[127., 133., 134.,  ..., 106.,  99., 118.],
        [156., 204., 100.,  ...,  19.,   0.,   7.],
        [131., 137., 134.,  ...,  81.,  64.,  92.],
        ...,
        [119., 107., 113.,  ...,  59.,  48.,  78.],
        [130., 121., 117.,  ..., 134., 165., 140.],
        [128., 127., 130.,  ..., 128., 128., 129.]], device='cuda:0')
tensor([[0.9117,

       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor(19421.6113, device='cuda:0', grad_fn=<MseLossBackward>)
19421.611328125
epoch : 20/20, recon loss = 19421.61132812


In [ ]:
for result in results:
    if result == [1,0,0,0]:
        print("Gryffindor")
    if result == [0,1,0,0]:
        print("Hufflepuff")
    if result == [0,0,1,0]:
        print("Ravenclaw")
    if result == [0,0,0,1]:
        print("Slytherin")